# this is a ipython notebook for assignment in CP course
# divert NFA(both ok with epsilon or not) to DFA

In [15]:
from queue import Queue

In [16]:
class graph:
    def __init__(self, _from : str, _to : str, _symbol : str):
        self._from = _from
        self._to = _to
        self._symbol = _symbol
    def show(self):
        print(str(self._from) + ' ' + str(self._symbol) + ' ' + str(self._to))

In [17]:
class NFA:
    '''
    @state: set of states in NFA
    @start_states: set of start states in NFA
    @terminal_states: set of terminal states in NFA
    @symbols: set of action symbols in NFA
    @mat: list of graph in NFA
    note: in this case symbol # means epsilon, not num of sth
    '''
    def __init__(self):
        self.states = set()
        self.start_states = set()
        self.terminal_states = set()
        self.symbols = set()
        self.mat = []
        
    def get_terminal_states(self):
        return self.terminal_states
        
    def get_start_states(self):
        return self.start_states
    
    def get_symbols(self):
        return self.symbols
        
    #show the content of NFA
    def show_NFA(self):
        for m in self.mat:
            m.show()
        print("symbols: " + str(self.symbols))
        print("states: " + str(self.states))
        print("start states: " + str(self.start_states))
        print("terminal states: " + str(self.terminal_states))
        
    #read NFA in specific formation
    def read_NFA(self, _string, _start_states, _terminal_states):
        graphs = _string.split('|')
        for _graph in graphs:
            _from, _sym, _to = _graph.split(' ')
            self.mat.append(graph(_from, _to, _sym))
            self.states.add(_from)
            self.states.add(_to)
            self.symbols.add(_sym)
        self.start_states = set(_start_states.split(' '))
        self.terminal_states = set(_terminal_states.split(' '))
        #raise NotImplementedError()

    def move(self, states, action):
        _set = set()
        for i in self.mat:
            if i._from in states and i._symbol == action and i._to not in _set:
                _set.add(i._to)
        return _set
        #raise NotImplementedError() 
        
    def get_epsilon_closure(self, states):
        _set = states.copy()
        stop_cue = len(_set)
        i = 0
        while i < stop_cue:
            prev_len = len(_set)
            for m in self.mat:
                if m._from in _set and m._symbol == '#' and m._to not in _set:
                    _set.add(m._to)
                    stop_cue += 1
            i += 1
            if prev_len == len(_set):
                break;
        return _set
        #raise NotImplementedError() 

In [18]:
class DFA:
    '''
    same as CLASS NFA
    '''
    def __init__(self):
        self.states = set()
        self.start_states = set()
        self.terminal_states = set()
        self.symbols = set()
        self.mat = []
        
    #read DFA in specific formation
    def read_DFA(self, _string, _start_states, _terminal_states):
        graphs = _string.split('|')
        for _graph in graphs:
            _from, _sym, _to = _graph.split(' ')
            self.mat.append(graph(_from, _to, _sym))
            self.states.add(_from)
            self.states.add(_to)
            self.symbols.add(_sym)
        self.start_states = set(_start_states.split(' '))
        self.terminal_states = set(_terminal_states.split(' '))
        
    #show the content of DFA
    def show_DFA(self):
        for m in self.mat:
            m.show()
        print("symbols: " + str(self.symbols))
        print("states: " + str(self.states))
        print("start states: " + str(self.start_states))
        print("terminal states: " + str(self.terminal_states))
                                        
    def get_DFA_from_NFA(self, _nfa : NFA):
        '''
        '''
        q = Queue(maxsize=0)
        T = {}
        nfa_terminal_states = _nfa.get_terminal_states()
        nfa_symbols = _nfa.get_symbols()
        index = 1
        cur_index = 0
        T["0"] = _nfa.get_epsilon_closure(_nfa.get_start_states())
        q.put(T["0"])
        while 1:
            if q.empty():
                break;
            prev_len = len(T)
            cur = q.get()
            for i in nfa_symbols:
                if i != '#':
                    t = _nfa.get_epsilon_closure(_nfa.move(cur, i))
                    print(str(cur) + " " + i + " " + str(t))
                    if len(t) == 0:
                        self.mat.append(graph(str(cur_index), '#', i))
                    elif t in T.values():
                        for j in range(0, index):
                            if t == T[str(j)]:
                                self.mat.append(graph(str(cur_index), str(j), i))
                                break;
                    else:
                        q.put(t)
                        T[str(index)] = t
                        self.mat.append(graph(str(cur_index), str(index), i))
                        index += 1
            cur_index += 1
        for key, value in T.items():
            t = value & nfa_terminal_states
            if len(t) > 0:
                self.terminal_states.add(key)
        self.start_states = {'0'}
        self.states = set(T.keys())
        self.symbols = nfa_symbols.copy()
        return T
        #raise NotImplementedError() 

# verification block
## using examples from textbook and course slide

In [19]:
#textbook example P51 例3.8
#input_NFA_string = "0 # 1|0 # 7|1 # 4|1 # 2|2 a 3|3 # 6|4 b 5|5 # 6|6 # 1|6 # 7|7 a 8|8 b 9|9 b 10"
#input_NFA_terminal_states = "10"
#input_NFA_start_states = "0"

#slide example
input_NFA_string = "p 0 q|p 0 s|p 1 q|q 1 q|q 0 r|q 1 r|r 1 p|r 0 s|s 1 p"
input_NFA_terminal_states = "p"
input_NFA_start_states = "p"

input_DFA_string = "0 a 1|1 a 2"
input_DFA_terminal_states = "1 2"
input_DFA_start_states = "0"

In [20]:


nfa = NFA()
nfa.read_NFA(input_NFA_string, input_NFA_start_states, input_NFA_terminal_states)
nfa.show_NFA()

dfa = DFA()
#dfa.read_DFA(input_DFA_string, input_DFA_start_states, input_DFA_terminal_states)
#dfa.show_DFA()

#print('#' * 50 + "\nverify get_epsilon_closure function")
#print(nfa.get_epsilon_closure(nfa.move(nfa.get_epsilon_closure(nfa.start_states), 'a')))
#print(' ' * 50)

#print('#' * 50 + "\nverify move function")
#print(nfa.move(nfa.get_epsilon_closure(nfa.start_states), 'a'))
#print(' ' * 50)

print('#' * 50 + "\nverify get_DFA_from_NFA function")
test_dict = dfa.get_DFA_from_NFA(nfa)
dfa.show_DFA()
print(' ' * 50)

p 0 q
p 0 s
p 1 q
q 1 q
q 0 r
q 1 r
r 1 p
r 0 s
s 1 p
symbols: {'1', '0'}
states: {'q', 's', 'r', 'p'}
start states: {'p'}
terminal states: {'p'}
##################################################
verify get_DFA_from_NFA function
{'p'} 1 {'q'}
{'p'} 0 {'q', 's'}
{'q'} 1 {'q', 'r'}
{'q'} 0 {'r'}
{'q', 's'} 1 {'q', 'r', 'p'}
{'q', 's'} 0 {'r'}
{'q', 'r'} 1 {'q', 'r', 'p'}
{'q', 'r'} 0 {'s', 'r'}
{'r'} 1 {'p'}
{'r'} 0 {'s'}
{'q', 'r', 'p'} 1 {'q', 'r', 'p'}
{'q', 'r', 'p'} 0 {'q', 's', 'r'}
{'s', 'r'} 1 {'p'}
{'s', 'r'} 0 {'s'}
{'s'} 1 {'p'}
{'s'} 0 set()
{'q', 's', 'r'} 1 {'q', 'r', 'p'}
{'q', 's', 'r'} 0 {'s', 'r'}
0 1 1
0 0 2
1 1 3
1 0 4
2 1 5
2 0 4
3 1 5
3 0 6
4 1 0
4 0 7
5 1 5
5 0 8
6 1 0
6 0 7
7 1 0
7 0 #
8 1 5
8 0 6
symbols: {'1', '0'}
states: {'1', '6', '8', '3', '7', '0', '4', '5', '2'}
start states: {'0'}
terminal states: {'5', '0'}
                                                  


In [21]:
test_dict

{'0': {'p'},
 '1': {'q'},
 '2': {'q', 's'},
 '3': {'q', 'r'},
 '4': {'r'},
 '5': {'p', 'q', 'r'},
 '6': {'r', 's'},
 '7': {'s'},
 '8': {'q', 'r', 's'}}